## Klasifikasi Warna Apel

---

Program ini bertujuan untuk mengklasifikasikan warna pada apel. Adapun model yang digunakan yakni YOLOv5

---

*   **Repo Link** : https://github.com/HeartFire01/applecolorclassify

In [1]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Deklarasi fungsi untuk klasifikasi warna
def classify_apple_color(cropped_apple):
    
    hsv_image = cv2.cvtColor(cropped_apple, cv2.COLOR_RGB2HSV)

    # Range warna untuk merah, kuning, dan hijau dalam ruang warna HSV
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])

    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([40, 255, 255])

    # Range hijau diperbarui untuk mendeteksi lebih banyak variasi hijau
    lower_green = np.array([35, 40, 40])  
    upper_green = np.array([90, 255, 255])

    # Membuat mask untuk masing-masing warna
    red_mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
    red_mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)

    # Mengecek area dari setiap masker untuk menentukan warna
    red_area = np.sum(red_mask1) + np.sum(red_mask2)
    yellow_area = np.sum(yellow_mask)
    green_area = np.sum(green_mask)

    # Menentukan warna berdasarkan area masker yang terbesar
    if green_area > red_area and green_area > yellow_area:
        return 'green'
    elif red_area > yellow_area:
        return 'red'
    else:
        return 'yellow'

### Load model YOLOv5

In [2]:

# Memuat model YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Memuat citra
image_path = '/home/purba/ai_assignment_20230726_classify.jpeg'  # Ganti dengan path gambar Anda
img = cv2.imread(image_path)

# Melakukan inferensi menggunakan model YOLOv5
results = model(img)

Using cache found in /home/purba/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-1-16 Python-3.9.20 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/home/purba/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


### Pembuatan bounding box dan labels

In [3]:

boxes = results.xywh[0][:, :-1].cpu().numpy()  
labels = results.xywh[0][:, -1].cpu().numpy()  
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

output_images = []

for i, (box, label) in enumerate(zip(boxes, labels)):

    x_center, y_center, w, h = box[:4]

    x1 = int(x_center - w / 2)
    y1 = int(y_center - h / 2)
    

    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = int(x1 + w)
    y2 = int(y1 + h)

    
    cropped_apple = img_rgb[y1:y2, x1:x2]


    apple_color = classify_apple_color(cropped_apple)

   
    output_filename = f"{apple_color}_{i+1}.jpg"
    output_images.append(output_filename)
    cv2.imwrite(output_filename, cv2.cvtColor(cropped_apple, cv2.COLOR_RGB2BGR))


    plt.imshow(cropped_apple)
    plt.axis('off')
    plt.title(f"{apple_color}_{i+1}")
    plt.show()

print("Gambar apel yang telah dipotong disimpan:", output_images)


Gambar apel yang telah dipotong disimpan: ['yellow_1.jpg', 'yellow_2.jpg', 'red_3.jpg', 'red_4.jpg', 'red_5.jpg', 'yellow_6.jpg', 'yellow_7.jpg', 'yellow_8.jpg', 'yellow_9.jpg', 'yellow_10.jpg', 'red_11.jpg', 'red_12.jpg', 'yellow_13.jpg', 'yellow_14.jpg', 'red_15.jpg', 'red_16.jpg', 'yellow_17.jpg', 'yellow_18.jpg', 'red_19.jpg', 'yellow_20.jpg', 'yellow_21.jpg', 'red_22.jpg']
